In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
#jd = pd.read_csv('job_data_dict_30_sample.csv')
#jd = pd.read_csv('job_data_dict_sal_weighted_30_sample.csv')
#jd = pd.read_csv('job_data_dict_all_sal_weighted.csv')
jd = pd.read_csv('job_data_dict_all_sal_weighted_major.csv')

In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9716 entries, 0 to 9715
Data columns (total 12 columns):
Unnamed: 0          9716 non-null int64
major_des           9716 non-null object
jobs                9716 non-null object
jobtitle            9716 non-null object
onetcode            9716 non-null object
major_code          9716 non-null int64
dictionary          9716 non-null object
p_an                9716 non-null float64
p_an_normal         9716 non-null float64
lowct               9716 non-null int64
major_dictionary    9716 non-null object
uid                 9716 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 986.8+ KB


In [4]:
jd.head()

,Unnamed: 0,major_des,jobs,jobtitle,onetcode,major_code,dictionary,p_an,p_an_normal,lowct,major_dictionary,uid
0,2,Management,computer and information systems managers,director of it planning development deployment,11-3021.00,11,director planning development deployment direc...,110000.0,0.097679,0,computer information systems managers computer...,1
1,6,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,45000.0,0.031779,0,financial managers branch department financial...,2
2,16,Management,human resources managers,human resources manager,11-3040.00,11,human resources manager human resources manage...,91197.5,0.078616,0,human resources managers human resources manag...,3
3,28,Management,treasurers and controllers,corporate controller,11-3031.01,11,corporate controller corporate controller year...,115000.0,0.102748,0,treasurers controllers treasurers controllers ...,4
4,31,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,82500.0,0.069798,0,financial managers branch department financial...,5


In [5]:

cat_columns = ['major_des', 'jobs','jobtitle','dictionary','major_dictionary', 'p_an_normal', 'lowct']
major_feat_col = ['major_dictionary', 'p_an_normal']   
feat_col = ['dictionary', 'p_an_normal']
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,major_des,jobs,jobtitle,dictionary,major_dictionary,p_an_normal,lowct
0,Management,computer and information systems managers,director of it planning development deployment,director planning development deployment direc...,computer information systems managers computer...,0.097679,0
1,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.031779,0
2,Management,human resources managers,human resources manager,human resources manager human resources manage...,human resources managers human resources manag...,0.078616,0
3,Management,treasurers and controllers,corporate controller,corporate controller corporate controller year...,treasurers controllers treasurers controllers ...,0.102748,0
4,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.069798,0


In [6]:
jd_2.major_des.unique()

array(['Management', 'Business and Financial Operations',
       'Computer and Mathematical', 'Architecture and Engineering',
       'Life, Physical, and Social Science',
       'Community and Social Service', 'Legal',
       'Education, Training, and Library',
       'Arts, Design, Entertainment, Sports, and Media',
       'Healthcare Practitioners and Technical', 'Healthcare Support',
       'Protective Service', 'Food Preparation and Serving Related',
       'Building and Grounds Cleaning and Maintenance',
       'Personal Care and Service', 'Sales and Related',
       'Office and Administrative Support',
       'Farming, Fishing, and Forestry', 'Construction and Extraction',
       'Installation, Maintenance, and Repair', 'Production',
       'Transportation and Material Moving'], dtype=object)

In [7]:
# subset the data to the major groups
jd_3 = jd_2[jd_2.major_des=='Transportation and Material Moving']
print(len(jd_3))
jd_3 = jd_3[jd_3.lowct==0]
len(jd_3)
jd_3.jobs

161


7988                         laborers and material movers
7989    supervisors of transportation machine and vehi...
7990                         laborers and material movers
7991               industrial truck and tractor operators
7993               industrial truck and tractor operators
7994               industrial truck and tractor operators
7995                         laborers and material movers
7996    supervisors of transportation machine and vehi...
7998    supervisors of transportation machine and vehi...
7999    supervisors of transportation machine and vehi...
8000    supervisors of transportation machine and vehi...
8001                         laborers and material movers
8002                         laborers and material movers
8004               industrial truck and tractor operators
8007               industrial truck and tractor operators
...
8123                         laborers and material movers
8125                         laborers and material movers
8128    su

In [8]:
jd_3.jobs.value_counts().min()

24

In [9]:
#df_numerical = jd_2.copy()
#encoder_dict = {}

#for col in string_variables:
#    le = preprocessing.LabelEncoder()
#    le.fit(jd_2[col])
#    encoder_dict[col] = le
#    df_numerical[col] = le.transform(jd_2[col])

#df_numerical.head()

In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_3[feat_col], jd_3['jobs'], random_state=1)

In [11]:
#feat = list(df_numerical.dictionary.values.flatten())
#type(feat)
#feat = unicode(feat, 'utf-8')

In [12]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
#y_test


In [13]:
X_test.shape

(28L, 2L)

In [14]:
X_train.shape

(84L, 2L)

In [15]:
type(X_train)

numpy.ndarray

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [17]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train[:,0])
train_dtm

<84x335 sparse matrix of type '<type 'numpy.int64'>'
	with 771 stored elements in Compressed Sparse Row format>

In [18]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test[:,0])
type(test_dtm)

scipy.sparse.csr.csr_matrix

In [19]:
train_features = vect.get_feature_names()
len(train_features)

335

In [20]:
train_features = train_features + list('s')

In [21]:
len(train_features)

336

In [22]:
train_sal = np.array(X_train[:,1])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [23]:
test_sal = np.array(X_test[:,1])
test_sal = test_sal[...,None]
type(test_sal)

numpy.ndarray

In [24]:
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [25]:
test_dtm_2 = np.array(test_dtm.todense())

In [26]:
test_dtm_2.shape

(28L, 335L)

In [27]:
train_dtm_2.shape

(84L, 335L)

In [28]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [29]:
type(train_dtm2)
train_dtm2.shape

(84L, 336L)

In [30]:
test_dtm2 = np.append(test_dtm_2, test_sal, 1)

In [31]:
# convert train_dtm to a regular array
#train_arr = train_dtm2.toarray()
#train_arr

In [34]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_dtm2, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
320,128,warehouse
110,58,forklift
192,56,operator
171,47,material
327,46,worker
122,45,handler
263,26,shipping
15,23,associate
146,22,labor
147,20,laborer


In [35]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, y_train)

C:\Anaconda\lib\site-packages\sklearn\naive_bayes.py:427: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  self.feature_count_ += safe_sparse_dot(Y.T, X)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm2)
preds

array(['laborers and material movers', 'laborers and material movers',
       'industrial truck and tractor operators',
       'industrial truck and tractor operators',
       'industrial truck and tractor operators',
       'laborers and material movers',
       'supervisors of transportation machine and vehicle operators',
       'supervisors of transportation machine and vehicle operators',
       'laborers and material movers', 'laborers and material movers',
       'laborers and material movers',
       'industrial truck and tractor operators',
       'laborers and material movers', 'laborers and material movers',
       'laborers and material movers',
       'supervisors of transportation machine and vehicle operators',
       'laborers and material movers', 'laborers and material movers',
       'industrial truck and tractor operators',
       'supervisors of transportation machine and vehicle operators',
       'industrial truck and tractor operators',
       'industrial truck 

In [37]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.964285714286
[[ 9  1  0]
 [ 0 13  0]
 [ 0  0  5]]


In [36]:
# predict (poorly calibrated) probabilities and calculate AUC
#probs = nb.predict_proba(test_dtm2)[:, 1]
#probs
#print(metrics.roc_auc_score(y_test, probs))

In [37]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [38]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [33]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

industrial truck and tractor operators       1.00      0.90      0.95        10
laborers and material movers       0.76      1.00      0.87        13
supervisors of transportation machine and vehicle operators       1.00      0.40      0.57         5

avg / total       0.89      0.86      0.84        28



In [32]:
# use SVM
from sklearn import svm
from sklearn import metrics
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm2, y_train)
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm2)

print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.857142857143
[[ 9  1  0]
 [ 0 13  0]
 [ 0  3  2]]


In [40]:
#from sklearn.cross_validation import cross_val_score
#cross_val_score(nb, train_dtm, y_train, cv=5, scoring='accuracy').mean()

In [41]:
# try using all of the data

In [42]:
train_dtm = vect.fit_transform(jd_3['dictionary'])
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [43]:
train_sal = np.array(jd_3['p_an_normal'])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [44]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [45]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, jd_3['jobs'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [46]:
preds = nb.predict(train_dtm2)
preds.shape

(128L,)

In [47]:

print(classification_report(jd_3['jobs'], preds) )

             precision    recall  f1-score   support

medical records and health information technicians       1.00      1.00      1.00        38
pharmacy technicians       1.00      1.00      1.00        41
registered nurses       1.00      1.00      1.00        49

avg / total       1.00      1.00      1.00       128



In [48]:

target_predicted = nb.predict(train_dtm2)
target_prob = nb.predict_proba(train_dtm2)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_3['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,medical records and health information technicians,pharmacy technicians,registered nurses,All
True,,,,
medical records and health information technicians,38,0,0,38
pharmacy technicians,0,41,0,41
registered nurses,0,0,49,49
All,38,41,49,128


In [49]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, jd_3['jobs'], cv=5, scoring='accuracy').mean()

0.95161538461538464